In [1]:
#--build a fake news classifier with NLP -> use tfidf as vectorizer
#--train on kaggle fake news dataset

#--read and understand the dataset
#--preprocess text data within dataframe
#--prepare train/test dataset
#--buil -> train -> evaluate model

In [2]:
#--mount gdrive
from google.colab import drive
drive.mount('/content/mount_gdrive')

Mounted at /content/mount_gdrive


In [3]:
#--import nltk library
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [5]:
#--import random libraries
import pandas as pd
from tqdm import tqdm

In [38]:
#--import nltk libraries
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
#--read and understand the dataset
data_path="/content/mount_gdrive/MyDrive/Learning_AI_dataset/NLP_dataset/dataset/kaggle_fake_news/train.csv"
df=pd.read_csv(data_path)
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [30]:
print(df.shape)

(20800, 5)


In [31]:
#--remove nan values
df.dropna(inplace=True)
print(df.shape)

(18285, 5)


In [32]:
#--initiate lemmatizer
lemmatizer=WordNetLemmatizer()

In [33]:
#--text preprocessing on selected columns

#--remove unwanted words
#--convert to lower case
#--convert each sent to list of words
#--remove stop words
#--apply stemming or lemmatization
#--modify within dataframe


def df_text_preprocessing(df,col_interest):

  df[col_interest]=df[col_interest].apply(lambda x:x.str.lower())

  for col in col_interest:
    df[col].replace('[^a-zA-Z]',' ',regex=True,inplace=True)

    sent_list=list(df[col])

    for i in tqdm(range(len(sent_list))):
      words=sent_list[i].split()
      review=[lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]

      review=' '.join(review)
      sent_list[i]=review

    df[col]=sent_list

  return df

In [34]:
#--intereset col
col_interest=['text']
filtered_df=df_text_preprocessing(df,col_interest)

100%|██████████| 18285/18285 [30:24<00:00, 10.02it/s]


In [36]:
filtered_df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide even see comey letter jason cha...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,ever get feeling life circle roundabout rather...,0


In [40]:
#---convert into vectors
sent_list=list(filtered_df['text'])

tfidf_vectorizer=TfidfVectorizer(max_features=4000,ngram_range=(1,3))
vectors=tfidf_vectorizer.fit_transform(sent_list).toarray()

In [41]:
#--prepare X,y
X=vectors
y=filtered_df['label']

In [42]:
#--prepare train and test dataset
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

In [44]:
#--build and train model
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()

model.fit(X_train,y_train)

MultinomialNB()

In [45]:
#--evaluate the model
preds=model.predict(X_test)

In [46]:
#--check the accuracy
from sklearn.metrics import confusion_matrix,accuracy_score
acc=accuracy_score(preds,y_test)
print("accuracy :",acc)

accuracy : 0.8911775428363106
